In [12]:
from lxml import html  
import csv,os,json
import requests
import pandas as pd
from time import sleep
#import socket
from selenium import webdriver
#import socks

In [18]:
#socks.setdefaultproxy(proxy_type=socks.PROXY_TYPE_SOCKS5, addr="127.0.0.1", port=9050)
#socket.socket = socks.socksocket
#driver = webdriver.Firefox(executable_path=r'C:\Geckodriver\geckodriver.exe')
#driver = webdriver.Chrome(executable_path=r'C:\Geckodriver\chromedriver.exe')
#driver.set_page_load_timeout(100)
#print(requests.get("http://icanhazip.com").text)
#print(driver.get("http://icanhazip.com").text)
#headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.90 Safari/537.36'}
driver = webdriver.Firefox(executable_path=r'C:\Geckodriver\geckodriver.exe')
driver.set_page_load_timeout(100)

In [31]:
def AmzonParser(url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    driver.get(url)
    #print(driver)
    rpage = requests.get(url,headers=headers)
    page = driver.page_source
    while True:
        sleep(3)
        try:
            doc = html.fromstring(page)
            XPATH_NAME = '//h1[@id="title"]//text()'
            #XPATH_SALE_PRICE = '//span[contains(@id,"ourprice") or contains(@id,"saleprice")]/text()'
            XPATH_SALE_PRICE = '//span[contains(@class,"offer-price") and contains(@class,"a-size-medium")]//text()'
            #XPATH_ORIGINAL_PRICE = '//td[contains(text(),"List Price") or contains(text(),"M.R.P") or contains(text(),"Price")]/following-sibling::td/text()'
            XPATH_CATEGORY = '//a[@class="a-link-normal a-color-tertiary"]//text()'
            XPATH_AVAILABILITY = '//div[@id="availability"]//text()' 
            XPATH_AUTHOR = '//a[contains(@class,"contributorNameID")]//text()'
            XPATH_ISBN10 = '//li/b[contains(text(),"ISBN-10:")]/following-sibling::text()'
            XPATH_PAGES = '//li/b[contains(text(),"Paperback") or contains(text(),"Hardcover") or contains(text(),"Print Length")]/following-sibling::text()'
            XPATH_PUBLISHER = '//li/b[contains(text(),"Publisher")]/following-sibling::text()'
            XPATH_LANGUAGE = '//li/b[contains(text(),"Language")]/following-sibling::text()'
            XPATH_DIMENSIONS = '//li/b[contains(text(),"Dimensions")]/following-sibling::text()'
            XPATH_WEIGHT = '//li/b[contains(text(),"Weight")]/following-sibling::text()'
            XPATH_RATING = '//li/b[contains(text(),"Average Customer Review:")]/following-sibling::span/span/a/i/span/text()'
           
            RAW_NAME = doc.xpath(XPATH_NAME)
            RAW_SALE_PRICE = doc.xpath(XPATH_SALE_PRICE)
            RAW_CATEGORY = doc.xpath(XPATH_CATEGORY)
            ##RAW_ORIGINAL_PRICE = doc.xpath(XPATH_ORIGINAL_PRICE)
            RAw_AVAILABILITY = doc.xpath(XPATH_AVAILABILITY)
            RAW_AUTHOR = doc.xpath(XPATH_AUTHOR)
            RAW_ISBN10 = doc.xpath(XPATH_ISBN10)
            RAW_PAGES = doc.xpath(XPATH_PAGES)
            RAW_PUBLISHER = doc.xpath(XPATH_PUBLISHER) 
            RAW_LANGUAGE = doc.xpath(XPATH_LANGUAGE)
            RAW_DIMENSIONS = doc.xpath(XPATH_DIMENSIONS)
            RAW_WEIGHT = doc.xpath(XPATH_WEIGHT)
            RAW_RATING = doc.xpath(XPATH_RATING)
            
            print(RAW_RATING)
 
            NAME = ' '.join(''.join(RAW_NAME).split()) if RAW_NAME else None
            SALE_PRICE = ' '.join(''.join(RAW_SALE_PRICE).split()).strip() if RAW_SALE_PRICE else None
            CATEGORY = ' > '.join([i.strip() for i in RAW_CATEGORY]) if RAW_CATEGORY else None
            #ORIGINAL_PRICE = ''.join(RAW_ORIGINAL_PRICE).strip() if RAW_ORIGINAL_PRICE else None
            AVAILABILITY = ''.join(RAw_AVAILABILITY).strip() if RAw_AVAILABILITY else None
            AUTHOR = ''.join(RAW_AUTHOR).strip() if RAW_AUTHOR else None
            ISBN10 = ''.join(RAW_ISBN10).strip() if RAW_ISBN10 else None
            PAGES = ''.join(RAW_PAGES).strip() if RAW_PAGES else None
            PUBLISHER = ''.join(RAW_PUBLISHER).strip() if RAW_PUBLISHER else None
            LANGUAGE = ''.join(RAW_LANGUAGE).strip() if RAW_LANGUAGE else None
            DIMENSIONS = ''.join(RAW_DIMENSIONS).strip() if RAW_DIMENSIONS else None
            WEIGHT = ''.join(RAW_WEIGHT).strip().strip('()').strip() if RAW_WEIGHT else None
            RATING = ''.join(RAW_RATING).strip() if RAW_RATING else None
            #if not ORIGINAL_PRICE:
                #ORIGINAL_PRICE = SALE_PRICE
 
            if rpage.status_code!=200 or not NAME or NAME == "":
                raise ValueError('captha')
            data = {
                    'NAME':NAME,
                    'SALE_PRICE':SALE_PRICE,
                    'CATEGORY':CATEGORY,
                    #'ORIGINAL_PRICE':ORIGINAL_PRICE,
                    'AVAILABILITY':AVAILABILITY,
                    'URL':url,
                    'AUTHOR' : AUTHOR,
                    'ISBN10' : ISBN10,
                    'PAGES' : PAGES,
                    'PUBLISHER' : PUBLISHER,
                    'LANGUAGE' : LANGUAGE,
                    'DIMENSIONS' : DIMENSIONS,
                    'WEIGHT' : WEIGHT,
                    'RATING' : RATING,
                    }
 
            return data
        except Exception as e:
            print(e)
            driver.get(url)
            #print(driver)
            rpage = requests.get(url,headers=headers)
            page = driver.page_source
 

In [32]:
def ReadAsin(AsinList):
    # AsinList = csv.DictReader(open(os.path.join(os.path.dirname(__file__),"Asinfeed.csv")))
    extracted_data = []
    for i in AsinList:
        url = "http://www.amazon.com/gp/product/"+i
        print("Processing: "+url)
        extracted_data.append(AmzonParser(url))
        sleep(5)
    f=open('data.json','w')
    json.dump(extracted_data,f,indent=4)
 

In [33]:
formatted_asin = []
df = pd.read_csv("amazon_product_asins.csv")
for index, row in df.iterrows():
    if len(row[0]) < 8:
        asin = "000" + row[0]
        formatted_asin.append(asin)
    elif len(row[0]) < 9:
        asin = "00" + row[0]
        formatted_asin.append(asin)
    elif len(row[0]) < 10:
        asin = "0" + row[0]
        formatted_asin.append(asin)
    else:
        formatted_asin.append(row[0])

In [34]:
if __name__ == "__main__":
    ReadAsin(formatted_asin[3300:3500])

Processing: http://www.amazon.com/gp/product/1559363843
['4.3 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0156013983
['4.6 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1942645589
['4.1 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1439153663
['4.6 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1250120896
[]
captha
[]
captha
['4.3 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0440221668
['4.6 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1401232744
['4.5 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0345545990
[]
captha
[]
captha
[]
captha
['4.4 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1593073283
[]
captha
['4.8 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0786966157
['4.7 out of 5 stars']
Processing: http://www.amazon.com/gp/product/146545859X
[]
captha
['4.6 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0545825016
['4.5

['4.7 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0451205766
['4.8 out of 5 stars']
Processing: http://www.amazon.com/gp/product/006083756X
['4.3 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0547572484
['3.5 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1945572590
['4.9 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0393608530
['3.9 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0486280411
['4.7 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0060628626
['4.7 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0785814531
['4.6 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1416524525
['4.5 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1328995062
['4.0 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1617755885
['4.6 out of 5 stars']
Processing: http://www.amazon.com/gp/product/055357339X
['4.4 out of 5 stars']
Processing: http://www.amazon

['4.1 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1101871377
['4.7 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1619845121
['4.6 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0143124757
['4.4 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0062515675
['4.3 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1423607120
['4.5 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1455585327
['4.5 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0345348109
['4.7 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1476789649
['3.6 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0778330761
[]
Processing: http://www.amazon.com/gp/product/051514701X
['4.2 out of 5 stars']
Processing: http://www.amazon.com/gp/product/125011988X
['4.8 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0385337523
['4.6 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0399

['4.0 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1478923040
['4.5 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0830843329
['4.9 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1416586296
['4.6 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0060256532
['4.6 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0061054887
['4.3 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0486406512
['4.5 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1455559806
['4.4 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0997316403
['4.1 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1538759632
['3.8 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1451681755
['4.5 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0375704027
['4.3 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1561641162
['4.7 out of 5 stars']
Processing: http://www.amazon

Processing: http://www.amazon.com/gp/product/1501180495
['4.5 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1101985550
['4.5 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0758278454
['4.2 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1538744090
[]
captha
['4.1 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0440243696
[]
captha
[]
captha
['4.3 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0735212511
['4.5 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0399179917
[]
captha
[]
Processing: http://www.amazon.com/gp/product/0374203105
['4.1 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1475031904
['4.3 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0345543017
[]
captha
[]
captha
['4.3 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0060934344
['4.3 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1542045770
['3.9 out of 5 stars']
Processing: 